Transfer learning using cryptobert and roberta

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

/home/peter/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-09 10:57:39.264729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 10:57:39.726396: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-09 10:57:39.726427: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do no

In [ ]:
# !pip install torch

In [2]:
!pip install protobuf==3.20.0

## Step 1 - Importing a sample of some Bitcoin Tweet Data to begin analysing the model

In [ ]:
pwd

In [3]:
# 20230308 - test data from twitter api, from 20220624 to 20220812 
# data = pd.read_csv('~/code/giadapi/crypto/raw_data/tweets_2022_reduced.csv')
data = pd.read_csv('tweets_2022_reduced_preproc.csv')
# 20230307 - test data from kaggle
# data = pd.read_csv('~/code/giadapi/crypto/data/raw/bitcoin_tweets1000000.csv', nrows = 1000)
# "\\wsl.localhost\Ubuntu\home\peter\code\giadapi\crypto\kaggle-tweets.zip"

In [ ]:
data.head()

In [ ]:
# #only run it if the dataset is from Twitter API
# data['text'] = data[['tweet']]
# data['date'] = data[['created_at']]

In [ ]:
data = data[['text', 'datetime']]

In [ ]:
data['text'][103]

## Step 2 - Cleaning the data

In [ ]:
# I have changed this to remove more information

def preprocess_2(text):
    new_text = []
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        
        new_text.append(t)
    return " ".join(new_text)

sample_text = preprocess_2(data['text'][0])
sample_text

In [ ]:
#use the preprocess_2 to clean the data

data['process_text'] = data.text
# data['label_xlm'] = data.text
# data['score_xlm'] = data.text
# data['negative_xlm'] = data.text
# data['neutral_xlm'] = data.text
# data['positive_xlm'] = data.text

data['label_bert'] = data.text
# data['score_bert'] = data.text
data['negative_bert'] = data.text
data['neutral_bert'] = data.text
data['positive_bert'] = data.text
for i in range(len(data)):
    data['process_text'][i] = preprocess_2(data['text'][i])

In [ ]:
data = data[['datetime', 'process_text']]

In [ ]:
data.to_csv('tweets_2022_reduced_preproc.csv')

## Step 3 - analyse the language and sentiments by pretrained model

In [ ]:
!pyenv local crypto

In [ ]:
#Model 1a - Use the xlm model - https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment?text=go+up

# model_path_xlm = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
# sentiment_task_xlm = pipeline("sentiment-analysis", model=model_path_xlm, tokenizer=model_path_xlm)

In [ ]:
# test_sent = sentiment_task_xlm(sample_text)
# print(test_sent[0]['score'])
# print(test_sent[0]['label'])

In [ ]:
#Model 1b - Use the xlm model, with full score
# MODEL_xlm = f"{model_path_xlm}"
# tokenizer_xlm = AutoTokenizer.from_pretrained(MODEL_xlm)
# config_xlm = AutoConfig.from_pretrained(MODEL_xlm)

# # PT
# model_xlm = AutoModelForSequenceClassification.from_pretrained(MODEL_xlm)
# # model_xlm.save_pretrained(MODEL_xlm)

In [ ]:
# def scores_xlm(x):
#     encoded_input_xlm = tokenizer_xlm(x, return_tensors='pt')
#     output_xlm = model_xlm(**encoded_input_xlm)
#     scores_xlm = output_xlm[0][0].detach().numpy()
#     scores_xlm = softmax(scores_xlm) #1st score is negative, 2nd score is netural, 3rd score is positive
#     return scores_xlm

In [ ]:
# score = scores_xlm(sample_text)
# score

In [ ]:
#Model 2a - Use the CryptoBERT model - https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment?text=go+up

model_path_bert = "ElKulako/cryptobert"
sentiment_task_bert = pipeline("sentiment-analysis", model=model_path_bert, tokenizer=model_path_bert)

In [ ]:
test_sent = sentiment_task_bert(sample_text)
print(test_sent[0]['score'])
print(test_sent[0]['label'])

In [ ]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"{model_path_bert}"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
config_bert = AutoConfig.from_pretrained(MODEL_bert)

# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
# model_bert.save_pretrained(MODEL_bert)

In [ ]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [ ]:
score = scores_bert(sample_text)
score

In [ ]:
data

In [ ]:
for i in range(len(data)):
    sentiment_xlm = sentiment_task_xlm(data['process_text'][i])
    data['label_xlm'][i] = sentiment_xlm[0]['label']
#     data['score_xlm'][i] = sentiment_xlm[0]['score']
    score_xlm = scores_xlm(data['process_text'][i])
    data['negative_xlm'][i] = score_xlm[0]
    data['neutral_xlm'][i] = score_xlm[1]
    data['positive_xlm'][i] = score_xlm[2]
    
    sentiment_bert = sentiment_task_bert(data['process_text'][i])
    data['label_bert'][i] = sentiment_bert[0]['label']
#     data['score_bert'][i] = sentiment_bert[0]['score']
    score_bert = scores_bert(data['process_text'][i])
    data['negative_bert'][i] = score_bert[0]
    data['neutral_bert'][i] = score_bert[1]
    data['positive_bert'][i] = score_bert[2]

In [ ]:
data

## Step 4 - compare the model performance

In [ ]:
def standardise(x):
    if (x == 'Neutral'):
        return 'neutral'
    elif (x == 'Bullish'):
        return 'positive'
    else:
        return 'negative'
data["label_bert"] = data["label_bert"].apply(standardise)

In [ ]:
data

In [ ]:
data['equality'] = data['label_xlm'] == data['label_bert']

In [ ]:
data

In [ ]:
data['label_xlm'].value_counts()

In [ ]:
data['label_bert'].value_counts()

In [ ]:
data['equality'].value_counts()

## Step 5: Count the total number of labels/scores (positive, negative vs neutral) by date

In [ ]:
data['date'] = pd.to_datetime(data.date)
data['new_date'] = data['date'].dt.date

In [ ]:
data

In [ ]:
grouped_data = pd.DataFrame(data.groupby(['new_date'])['label_xlm'].value_counts().unstack().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

grouped_data_bert = pd.DataFrame(data.groupby(['new_date'])['label_bert'].value_counts().unstack().fillna(0).reset_index())
grouped_score = pd.DataFrame(data.groupby(['new_date'])[['negative_xlm', 'neutral_xlm', 'positive_xlm','negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())

grouped_data[['date', 'count_negative_xlm', 'count_neutral_xlm', 'count_positive_xlm']] = grouped_data[['new_date', 'negative', 'neutral', 'positive']]
grouped_data[['count_negative_bert', 'count_neutral_bert', 'count_positive_bert']] = grouped_data_bert[['negative', 'neutral', 'positive']]
grouped_data[['negative_xlm', 'neutral_xlm', 'positive_xlm','negative_bert', 'neutral_bert', 'positive_bert']] = grouped_score[['negative_xlm', 'neutral_xlm', 'positive_xlm','negative_bert', 'neutral_bert', 'positive_bert']]

grouped_data = grouped_data.drop(["new_date", 'negative', 'neutral', 'positive'], axis='columns')

In [ ]:
grouped_data

## Step 6 - Download the data

In [ ]:
start_date = grouped_data['date'][0]
start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
start_date_str

In [ ]:
end_date = grouped_data['date'][len(grouped_data)-1]
end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
end_date_str

In [ ]:
file_name = f"{start_date_str}_{end_date_str}_twitter_comments.csv"
data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [ ]:
file_name = f"{start_date_str}_{end_date_str}_twitter_transferlearning.csv"
grouped_data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")